## Домашнее задание

__Задание 1. Генератор случайных матриц.__

Реализовать генератор матриц, который должен поддерживать функции:
* Генерация абсолютно случайной матрицы $n\times m$
* Генерация случайной диагональной матрицы $n\times n$
* Генерация случайной верхнетреугольной матрицы
* Генерация случайной нижнетреугольной матрицы
* Генерация симметричной матрицы
* Генерация возмущения матрицы $n\times m$, каждый элемент которой не превосходит по модулю заданный $\varepsilon$. Оценить величину нормы матрицы возмущений в зависимости от параметра $\varepsilon$ (оценить верхную границу).

Оценить численно вероятность того, что созданная матрица будет вырожденной для какого-либо случая выше. #создать несколько матриц и посчитать отношение нулевых ко всем

In [2]:
#Письменные уражнения в пдф файле
#Ex_1
import numpy as np

def matrix_generate(rows, columns, type_ = "full", eps = 0):
    """
    matrix_generate(rows, columns, type_ = "full")

    Создаёт случайную матрицу выбранного типа.

    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".

    Parameters
    ----------

    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "diagonal", "upper_triangular", "lower_triangular", "symmetric",
        "singular" и т.д.
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц.

    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.

    Notes
    -----
    Поддерживаемые типы матриц:
        "full","diagonal","upper_triangular","lower_triangular",
        "symmetric", "singular",
        ...

    """

    A = None

    if type_ == "full":
        A = np.random.random(size=(rows, columns))

    elif type_ == "diagonal":
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.diag(np.random.rand(rows, columns))

    elif type_ == "upper_triangular":
        A = np.random.random(size=(rows, columns))
        A = np.triu(A)
                    
    # Для нижнетреугольной подумайте, как сделать без циклов for :) (звёздочка)
    elif type_ == "lower_triangular":
        A = np.random.random(size=(rows, columns))
        A = np.tril(A)
        
    # И эту секцию тоже перепишите без for (звёздочка). Учтите, что портить uniform распределение нельзя.
    elif type_ == "symmetric":
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.random.random(size=(rows, columns))
            A = A + A.T
            
    elif type_ == "singular":
        A = np.random.random(size=(rows, columns))
        A[:, -1] = A[:, 0] # делаем последний столбец равным первому
        A[:, np.random.permutation(A.shape[1])] #shuffle
        
    elif type_ == "step":
        A = np.triu(np.random.random((rows, rows)), 1) + np.eye(rows)
        indices = list(range(rows))
        np.random.shuffle(indices)
        A = np.delete(A, indices[:rows - columns], axis=0)
        A = np.concatenate((A, np.zeros((rows - columns, rows))), axis=0)
            
    elif type_ == "perturbation":
        A = np.random.uniform(-eps, eps, size=(rows, columns))

    return A

In [6]:
print(matrix_generate(1, 3),'\n')
print(matrix_generate(4, 4, type_ = "upper_triangular"),'\n')
print(matrix_generate(4, 3, type_ = "upper_triangular"),'\n')
print(matrix_generate(4, 4, type_ = "symmetric"),'\n')
print(matrix_generate(4, 3, type_ = "step"),'\n')
print(matrix_generate(4, 1, type_ = "symmetric"))

[[0.99299055 0.84952361 0.75750753]] 

[[0.9731102  0.85136072 0.05210937 0.60533974]
 [0.         0.55807543 0.69204736 0.33151945]
 [0.         0.         0.95481242 0.90199604]
 [0.         0.         0.         0.82268744]] 

[[0.1890788  0.68517515 0.98798629]
 [0.         0.75863602 0.8584227 ]
 [0.         0.         0.05338683]
 [0.         0.         0.        ]] 

[[1.48978979 1.86426383 1.25809502 0.99470763]
 [1.86426383 0.57076971 1.80853555 1.01935927]
 [1.25809502 1.80853555 1.5882462  1.61218491]
 [0.99470763 1.01935927 1.61218491 0.31395379]] 

[[1.         0.31714013 0.9980875  0.69606948]
 [0.         1.         0.66122144 0.95664497]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.        ]] 

Error with type symmetric and shape (4,1)


Оценка величины нормы матрицы возмущений:
$\|\mathbf{A}\|_{F} \leq\ \varepsilon\ \sqrt{nm}$

In [10]:
s = 0
for i in range(100000):
    det = np.linalg.det(matrix_generate(20, 20, type_='full'))
    if np.abs(det) < 1e-14:
        s+=1

print(f"вероятность получить вырожденную матрицу равна {s/100000}")

вероятность получить вырожденную матрицу равна 0.0


__Задание 2. Вычисление матричных норм и числа обусловленности.__

Реализовать вычисление трех основных норм векторов (L1, L2 и максимальную) и подчиненных им матричных норм. Реализовать вычисление числа обусловленности.

Примечание: для вычисления собственных значений можно использовать linalg.eigvals из модуля scipy.

In [7]:
#Exersice_2
from scipy.linalg import eigvals

def l1_norm(x):
    return np.sum(np.abs(x)),

def l2_norm(x):
    return np.sqrt(np.sum(x**2))

def max_norm(x):
    return np.max(np.abs(x))

def l1_norm_matrix(A):
    return np.max(np.sum(np.abs(A), axis=0))

def l2_norm_matrix(A):
    return np.sqrt(np.max(eigvals(np.array(A) @ np.array(A).T)))

def max_norm_matrix(A):
    return np.max(np.sum(np.abs(A), axis=1))

def cond_num(A, norm="L1"):
    if norm == "L1":
        return l1_norm_matrix(A) * l1_norm_matrix(np.linalg.inv(A))
    elif norm == "L2":
        return l2_norm_matrix(A) * l2_norm_matrix(np.linalg.inv(A))
    elif norm == "inf":
        return max_norm_matrix(A) * max_norm_matrix(np.linalg.inv(A))

__Задание 6*.  Тензорная свёртка.__

Рассмотрим функцию, отображающую шесть тензоров на один тензор: $Z\left(\lambda^{(1)}, \lambda^{(2)}, \lambda^{(3)}, \Gamma^{(1)}, \Gamma^{(2)}, U\right)$ :


$$
Z_{a h i j}=\sum_{b c d e f g} \lambda^{(1)}{ }_{a b} \Gamma_{c b d}^{(1)} \lambda^{(2)}{ }_{d e} \Gamma_{f e g}^{(2)} \lambda_{g h}^{(3)} U_{i j c f}
$$

редположив, что все индексы пробегают значения от 1 до χ, проведите эксперимент и сравните скорость
различных реализаций функции Z. Исследуйте значения χ в диапазоне 3–50.


- В файле convolution.ipynb вы можете найти релизацию глупого способа вычисления этой свертки, который требует $\chi^4 \times \chi^6=\chi^{10}$ операций. На самом деле это можно вычислить гораздо быстрее!
- С помощью функции numpy.einsum (нужно использовать аргумент optimize), можно добиться намного большей производительности. Чтобы понять, что происходит под капотом, воспользуйтесь функцией numpy.einsum_path. Какое минимальное количество операций требуется для вычисления $Z$ ?
- Посмотрев на вывод функции numpy.einsum_path, peализуйте алгоритм для вычисления $Z$, который столь же эффективен, как numpy.einsum, но использует более элементарные numpy.dot и numpy.tensor_dot.


In [ ]:
c=5
np.random.seed(42) #генерация данных
lambda1 = np.random.normal(size=(c, c))
lambda2 = np.random.normal(size=(c, c))
lambda3 = np.random.normal(size=(c, c))
G1 = np.random.normal(size=(c, c, c))
G2 = np.random.normal(size=(c, c, c))
U = np.random.normal(size=(c, c, c, c))

out = np.einsum_path("ab,cbd,de,feg,gh,ijcf->ahij", lambda1, G1, lambda2, G2, lambda3, U, optimize='optimal')
print(out[0], out[1])

In [ ]:
def Z_naive(lambda1, lambda2, lambda3, G1, G2, U):
    c = lambda1.shape[0]
    Z = np.zeros(shape=(c, c, c, c))
    for a, b, c, d, e, f, g, h, i, j in itertools.product(*([range(c)]*10)):
        Z[a, h, i, j] += lambda1[a, b]*lambda2[d, e]*lambda3[g, h]*G1[c, b, d]*G2[f, e, g]*U[i, j, c, f]
    return Z


def Z_optimal(lambda1, lambda2, lambda3, G1, G2, U):
    c = lambda1.shape[0]

    lG1 = np.tensordot(G1, lambda1, axes = ([1], [1]))
    lG2 = np.tensordot(G2, lambda2, axes = ([1], [1]))
    lG3 = np.tensordot(lG2, lambda3, axes = ([2], [0]))
    GG = np.tensordot(lG3, lG1, axes = ([0], [2]))
    Z = np.tensordot(GG, U, axes = ([1, 2], [2, 3]))
    
    return Z

In [ ]:
%%timeit
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)

In [ ]:
%%timeit
Z = Z_optimal(lambda1, lambda2, lambda3, G1, G2, U)

In [ ]:
%%timeit
Z = np.einsum("ab,cbd,de,feg,gh,ijcf->ahij", lambda1, G1, lambda2, G2, lambda3, U, optimize='optimal')
#нужно Х^6 операций для Z_optimal